In [1]:
import PI

meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 100,
    'is_flatten': False
}

PI = PI.PIInterface(meta_params)

import torch
import torch.nn as nn
import torch.optim as optim
from MNIST_models import *

model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
# print('train acc:', PI.eval_model('train'))
# print('test acc:', PI.eval_model('test'))

1000 100
(100, 1, 28, 28)


/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessi

In [2]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from scipy.ndimage.filters import gaussian_filter1d
from matplotlib import pyplot as plt
%matplotlib inline

prefixs = ['store_zero/', 'store_one/', 'store_two/', 'store_three/', 'store_four/', 
           'store_five/', 'store_six/', 'store_seven/', 'store_eight/', 'store_nine/']     

# LOAD 
# guards = []
# for idx, prefix in enumerate(prefixs):
#     # LOAD TRAINED SUB-GUARD
#     guard_model = torch.load('store_subs/'+prefix+'guard.pt')
#     guards.append(guard_model)

guards = []
for _ in range(10):
    gaurd_model = Guard()
    guards.append(gaurd_model)
    
    
prefix = 'store/'    
# LOAD 
adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
set_of_train_dataset, set_of_test_dataset = [], []

for adv_type in adv_types:
    # extract from the store file 
    if adv_type == 'None': fn_name=prefix+'normal.txt'
    else: fn_name=prefix+adv_type+'.txt'
    fp = open(fn_name, 'rb')
    set_of_signatures = pickle.load(fp)
    
    # separate and store in for later training and evaluation 
    if adv_type == 'None': split_percentage = 0.8
    else: split_percentage = 0.8
    split_line = int(len(set_of_signatures)*split_percentage)
    train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
    set_of_train_dataset.append(train_set_of_signatures)
    set_of_test_dataset.append(test_set_of_signatures)
    fp.close()
    print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))

None 1000 800 200
FGSM 813 650 163
JSMA 966 772 194
CWL2 877 701 176
LINFPGD 978 782 196
LINFBI 970 776 194
ENL1 1000 800 200
ST 991 792 199


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch

guard_model = AssemebleGuard(guards)
guard_model.train()
epoches = 10

train_accs, test_accs, losses, set_train_sub_accs, set_test_sub_accs = train_guard_model(guard_model, set_of_train_dataset, set_of_test_dataset, adv_types, epoches)
losses = [x.item() for x in losses]
guard_model.eval()
# torch.save(guard_model, 'store/guard.pt')
# guard_model = torch.load('store/guard.pt')

benign correct: 628 / 800
adv ( FGSM ) correct: 650 / 650
adv ( JSMA ) correct: 767 / 772
adv ( CWL2 ) correct: 660 / 701
adv ( LINFPGD ) correct: 773 / 782
adv ( LINFBI ) correct: 751 / 776
adv ( ENL1 ) correct: 762 / 800
adv ( ST ) correct: 777 / 792
acc: 0.949777704594105

benign correct: 153 / 200
adv ( FGSM ) correct: 163 / 163
adv ( JSMA ) correct: 191 / 194
adv ( CWL2 ) correct: 156 / 176
adv ( LINFPGD ) correct: 194 / 196
adv ( LINFBI ) correct: 191 / 194
adv ( ENL1 ) correct: 182 / 200
adv ( ST ) correct: 194 / 199
acc: 0.9356110381077529

epoch: 1 loss: 2332.85302734375
acc (train): 0.949777704594105
acc (test): 0.9356110381077529

one epoch training time in seconds 697.067898
benign correct: 699 / 800
adv ( FGSM ) correct: 650 / 650
adv ( JSMA ) correct: 771 / 772
adv ( CWL2 ) correct: 665 / 701
adv ( LINFPGD ) correct: 781 / 782
adv ( LINFBI ) correct: 770 / 776
adv ( ENL1 ) correct: 777 / 800
adv ( ST ) correct: 783 / 792
acc: 0.9708546023382183

benign correct: 162 / 200


In [ ]:
import copy
from scipy.ndimage.filters import gaussian_filter1d

smooth_train_accs = gaussian_filter1d(train_accs, sigma=1)
smooth_test_accs = gaussian_filter1d(test_accs, sigma=1)
smooth_losses = gaussian_filter1d(losses, sigma=1)
norm_smooth_losses = [float(i)/max(smooth_losses) for i in smooth_losses]

from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure()

# summarize history for accuracy
plt.plot(smooth_train_accs)
plt.plot(smooth_test_accs)
plt.plot(norm_smooth_losses)
plt.title('model accuracy and loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train acc', 'test acc', 'loss'], loc='lower right')
plt.ylim([0, 1.05])

# plt.savefig('outputs.jpg')
plt.show()

In [ ]:
test_guard_model(guard_model, set_of_test_dataset, adv_types)

In [ ]:
figsize = (12, 4)
fig=plt.figure(figsize=figsize)

num_of_epoches = len(set_train_sub_accs)
num_of_adv_types = len(set_train_sub_accs[0])
fig.add_subplot(1, 2, 1)
for i in range(num_of_adv_types):
    adv_type_accs = [sub_accs[i] for sub_accs in set_train_sub_accs]
    smooth_adv_type_accs = gaussian_filter1d(adv_type_accs, sigma=1)
    plt.plot(smooth_adv_type_accs)
    
# plt.legend(filtered_adv_types, loc='lower right')
plt.legend(adv_types, loc='lower right')
plt.title('train')
plt.ylim([0, 1.05])

num_of_epoches = len(set_test_sub_accs)
num_of_adv_types = len(set_test_sub_accs[0])
fig.add_subplot(1, 2, 2)
for i in range(num_of_adv_types):
    adv_type_accs = [sub_accs[i] for sub_accs in set_test_sub_accs]
    smooth_adv_type_accs = gaussian_filter1d(adv_type_accs, sigma=1)
    plt.plot(smooth_adv_type_accs)
    
plt.legend(adv_types, loc='lower right')
plt.title('test')
plt.ylim([0, 1.05])

plt.savefig('assemble_10_untrained_one_PPRD.jpg')
plt.show()

In [ ]:
# Compare assemble_10_untrained_one_PPRD and assemble_10_trained_one_PPRD